In [222]:
import numpy as np
from Reader_functions import *
from data_locator import *

In [223]:
#Looks for an input file by findning a txt file in the specified directory
#It checks through the files alphabetically and picks the last one it sees
search_directory = 'Examples\Ferrihydrite-formation_SIMULATIONS'
path = find_txt_file(search_directory)

In [224]:
with open(path, "r") as f:
    lines = f.readlines()

split_file = []
current_list = []

for line in lines:
    if line[0:3] == '***':
        split_file.append(current_list)
        current_list = []
    current_list.append(line.strip())
split_file.append(current_list)
split_file.pop(0)

if len(split_file) != 20:
    print(f'Error: Input file {path} doesn\'t match the expected format. Please ensure your input file is unmodified and that there are no other .txt files at the top level of your search directory.')

for item in split_file:
    print(item)

['*** TIME INFORMATION ***', 'Total time: 1.00E-2', 'dt: 1.00E-8', 'dt_spring: 1.00E-9', 'dt_data: 1.00E-4', 'dt_image: 1.00E-4', '']
['*** SYSTEM INFORMATION ***', 'L_x: 0.1', 'L_y: 0.1', 'L_z_out: 0.01', 'L_z_in: 0.09', 'Partition Nx: 10', 'Partition Ny: 10', 'Partition Nz: 10', '']
['*** MOLECULES ***', '', 'MOLECULE: "UnitCell" Intracellular Number 0 Site_Types 12 Total_Sites 12 Total_Links 31 is2D false', '{', 'TYPE: Name "Top" Radius 1.59690 D 1.000 Color RED STATES "st0"', 'TYPE: Name "Bottom" Radius 1.59690 D 1.000 Color GREEN STATES "St0"', 'TYPE: Name "Left" Radius 0.55010 D 1.000 Color MAROON STATES "St0"', 'TYPE: Name "Right" Radius 0.55010 D 1.000 Color PURPLE STATES "St0"', 'TYPE: Name "Front" Radius 0.55010 D 1.000 Color BLUE STATES "St0"', 'TYPE: Name "Back" Radius 0.55010 D 1.000 Color SLATE_BLUE STATES "St0"', 'TYPE: Name "Left_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES "NonOcc" "Occ"', 'TYPE: Name "Right_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES 

In [225]:
#Extract molecule data
molecules = []
molecule = []
for line in split_file[2]:
    if line[0:9] == 'MOLECULE:':
        molecules.append(molecule)
        molecule = []
    molecule.append(line)
molecules.append(molecule)    
molecules.pop(0)

for item in molecules:
    print(item)

['MOLECULE: "UnitCell" Intracellular Number 0 Site_Types 12 Total_Sites 12 Total_Links 31 is2D false', '{', 'TYPE: Name "Top" Radius 1.59690 D 1.000 Color RED STATES "st0"', 'TYPE: Name "Bottom" Radius 1.59690 D 1.000 Color GREEN STATES "St0"', 'TYPE: Name "Left" Radius 0.55010 D 1.000 Color MAROON STATES "St0"', 'TYPE: Name "Right" Radius 0.55010 D 1.000 Color PURPLE STATES "St0"', 'TYPE: Name "Front" Radius 0.55010 D 1.000 Color BLUE STATES "St0"', 'TYPE: Name "Back" Radius 0.55010 D 1.000 Color SLATE_BLUE STATES "St0"', 'TYPE: Name "Left_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES "NonOcc" "Occ"', 'TYPE: Name "Right_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES "NonOcc" "Occ"', 'TYPE: Name "Front_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES "NonOcc" "Occ"', 'TYPE: Name "Back_site" Radius 0.47210 D 1.000 Color LIGHT_GRAY STATES "NonOcc" "Occ"', 'TYPE: Name "Top_site" Radius 0.68680 D 1.000 Color LIGHT_GRAY STATES "Occ" "NonOcc"', 'TYPE: Name "Bottom_site" Radi

In [226]:
#Molecule Processing
multi_state_types = []
multi_state_sites = []
for item in molecules:   
    #General Info
    typStr = ''
    split_general_info = item[0].split()
    site_types = int(split_general_info[6])
    for j in range(2,2+site_types):
        type_split = item[j].split()
        type_name = type_split[2]
        states = [i for i in type_split if '"' in i]
        site_name = states.pop(0)
        typStrAddition = ''
        if len(states) == 1:
            pass
        else:
            for state in states:
                typStrAddition = typStrAddition + f'{state[1:-1]}, '
            typStrAddition = typStrAddition[:-2]
            typStrAddition = f' ({len(states)} possible states: {typStrAddition})'
            multi_state_types.append(type_name[1:-1])
            multi_state_sites.append(j-2)
        typStr = f'{typStr}{type_name[1:-1]}{typStrAddition}, '
    print(f'MOLECULE: {split_general_info[1]}\n')
    print(f'This molecule has {split_general_info[6]} site types: {typStr[:-2]}')

    #Sites
    total_sites = int(split_general_info[8])
    site_data = []
    for i in range(3 + site_types, 3 + site_types + 3*total_sites, 3):
        site_split1 = item[i].split()
        site_split2 = item[i+1].split()
        site = [f'{site_split1[0]} {site_split1[1]}', site_split1[3], site_split2[2][1:-1]]
        site_data.append(site)

    #Sites (default ordering)
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for site in site_data:
        print(f'{site[0]} ({site[1]}) of type "{site[2]}"')
        pass

    #Sites (grouped by type)
    '''
    sites_reordered = []
    site_type_names = []  
    for site in site_data:
        if site[2] not in site_type_names:
            site_type_names.append(site[2])
            #Site type name, Numpy Array(#Intracellular, #Extracellular, #Membrane), [list of sites] 
            sites_reordered.append([site[2], location2vec(site[1]), [site[0]]])
        else:
            for reordered_site in sites_reordered:
                if site[2] == reordered_site[0]:
                    reordered_site[1] = reordered_site[1] + location2vec(site[1])
                    reordered_site[2].append(site[0])
    
    print(f'\nThis molecule consists of {total_sites} connected sites:')
    for reordered_site in sites_reordered:
        str2 = 'Placeholder 2'
        location_list = list(reordered_site[1])
        site_list = list(reordered_site[2])
        for index in location_list:
            addStr = ''
            if location_list[0] != 0:
                addStr = addStr + f'{location_list[0]} intracellular, '
            if location_list[1] != 0:
                addStr = addStr + f'{location_list[1]} extracellular, '
            if location_list[2] != 0:
                addStr = addStr + f'{location_list[2]} membrane, '
        addStr2 = ''
        for index in site_list:
            addStr2 = addStr2 + index + ', '
        print(f'Type {reordered_site[0]}: {addStr[:-2]} sites ({addStr2[:-2]})')
    '''
    
    total_links = int(split_general_info[10])
    link_data = []
    for i in range(4 + site_types + 3*total_sites, 4 + site_types + 3*total_sites + total_links):
        split_link = item[i].split()
        link = [split_link[2], split_link[5]]
        link_data.append(link)
    
    #[[link index, list of links],[link index, list of links],...]
    link_lists = []
    for site in site_data:
        split_site = site[0].split()
        link_count_init = [split_site[1], []]
        link_lists.append(link_count_init)
    for link_list in link_lists:
        for link in link_data:
            if link_list[0] == link[0]:
                link_list[1].append(link[1])
            elif link_list[0] == link[1]:
                link_list[1].append(link[0])
            else:
                pass

    link_lists_remaining = link_lists

    #Links (no repeated entries)
    '''
    print(f'\nThis molecule had {total_links} total links between sites:')
    while True:
        record_length = 0
        for link_list in link_lists:
            if len(link_list[1]) > record_length:
                record_length = len(link_list[1])
                record_loc = int(link_list[0])
        if record_length == 0:
            break

        site_str = ''
        for site in link_lists_remaining[record_loc][1]:
            site_str = f'{site_str} site {site},'
        
        print(f'Site {record_loc} is connected to{site_str[:-1]}')
        for link_list_remaining in link_lists_remaining:
            link_list_remaining[1] = [i for i in link_list_remaining[1] if i != str(record_loc)] 
        link_lists_remaining[record_loc][1] = []
    print('\n')
    '''

    #All links for all sites
    print(f'\nThis molecule had {total_links} total links between sites:')
    for link in link_lists:
        site_str = ''
        for site in link[1]:
            site_str = f'{site_str} site {site},'
        print(f'Site {link[0]} is connected to{site_str[:-1]}')
    print('\n')

MOLECULE: "UnitCell"

This molecule has 12 site types: Top, Bottom, Left, Right, Front, Back, Left_site (2 possible states: NonOcc, Occ), Right_site (2 possible states: NonOcc, Occ), Front_site (2 possible states: NonOcc, Occ), Back_site (2 possible states: NonOcc, Occ), Top_site (2 possible states: Occ, NonOcc), Bottom_site (2 possible states: NonOcc, Occ)

This molecule consists of 12 connected sites:
SITE 0 (Intracellular) of type "Top"
SITE 1 (Intracellular) of type "Bottom"
SITE 2 (Intracellular) of type "Left"
SITE 3 (Intracellular) of type "Right"
SITE 4 (Intracellular) of type "Front"
SITE 5 (Intracellular) of type "Back"
SITE 6 (Intracellular) of type "Top_site"
SITE 7 (Intracellular) of type "Bottom_site"
SITE 8 (Intracellular) of type "Left_site"
SITE 9 (Intracellular) of type "Right_site"
SITE 10 (Intracellular) of type "Front_site"
SITE 11 (Intracellular) of type "Back_site"

This molecule had 31 total links between sites:
Site 0 is connected to site 2, site 4, site 3, sit

In [227]:
#Extract reaction data
transition_reactions = []
allosteric_reactions = []
binding_reactions = []
reaction_types = [['transition_reactions', 5], ['allosteric_reactions', 6], ['binding_reactions', 7]]
for reaction_type in reaction_types:
    for line in split_file[reaction_type[1]]:
        if line != '':
            exec(reaction_type[0] + '.append(line)')
    exec(reaction_type[0] + '.pop(0)')


In [228]:
print('Reaction Rules')
#State Transition Reactions
if len(transition_reactions) != 0:
    print(f'\nState Transition Reactions: {len(transition_reactions)}')
    for transition_reaction in transition_reactions:
        split_reaction = transition_reaction.split('\'')
        split_condition = split_reaction[10].split()
        condition = split_condition[-1]

        if split_reaction[13] not in multi_state_types:
            state = ''
        elif split_reaction[15] == 'Any_State':
            state = ' in any state'
        else:
            state = ' of state ' + split_reaction[15]
        
        if condition == 'None':
            condition_str = ''
        elif condition == 'Free':
            condition_str = f'provided that it is free'
        else:
            condition_str = f'provided that it is bound to site {split_reaction[13]} of a molecule {split_reaction[11]}{state}'
        print(f'A site of type {split_reaction[5]} in a {split_reaction[3]} molecule can change its state from {split_reaction[7]} to {split_reaction[9]} {condition_str}')

#Allosteric Reactions
if len(allosteric_reactions) != 0:
    print(f'\nAllosteric Reactions: {len(allosteric_reactions)}')
    for allosteric_reaction in allosteric_reactions:
        split_reaction = allosteric_reaction.split('\'')
        split_allosteric_site = split_reaction[-3].split()

        if int(split_allosteric_site[3]) not in multi_state_sites:
            condition = ''
        else:
            condition = f'provided that Site {split_allosteric_site[3]} of the same molecule is in state {split_reaction[-2]}'
        print(f'{split_reaction[4][3:-3]} of molecule {split_reaction[3]} can change its state from {split_reaction[5]} to {split_reaction[7]} {condition}')

#Biomolecular Binding Reactions
if len(binding_reactions) != 0:
    print(f'\nBiomolecular Binding Reactions: {len(binding_reactions)}')
    for binding_reaction in binding_reactions:
        split_reaction = binding_reaction.split('\'')
        if split_reaction[5] not in multi_state_types:
            state1 = ''
        elif split_reaction[7] == 'Any_State':
            state1 = ' (in any state)'
        else:
            state1 = f' (of state {split_reaction[7]})'
        if split_reaction[11] not in multi_state_types:
            state2 = ''
        elif split_reaction[13] == 'Any_State':
            state2 = ' (in any state)'
        else:
            state2 = f' (of state {split_reaction[13]})'
        print(f'Site {split_reaction[5]} of molecule {split_reaction[3]}{state1} can bind to site {split_reaction[11]} of {split_reaction[9]}{state2}')

Reaction Rules

State Transition Reactions: 6
A site of type Bottom_site in a UnitCell molecule can change its state from NonOcc to Occ provided that it is bound to site Bottom_site of a molecule UnitCell of state NonOcc
A site of type Top_site in a UnitCell molecule can change its state from NonOcc to Occ provided that it is bound to site Top_site of a molecule UnitCell of state NonOcc
A site of type Left_site in a UnitCell molecule can change its state from NonOcc to Occ provided that it is bound to site Left_site of a molecule UnitCell of state NonOcc
A site of type Right_site in a UnitCell molecule can change its state from NonOcc to Occ provided that it is bound to site Right_site of a molecule UnitCell of state NonOcc
A site of type Front_site in a UnitCell molecule can change its state from NonOcc to Occ provided that it is bound to site Front_site of a molecule UnitCell of state NonOcc
A site of type Back_site in a UnitCell molecule can change its state from NonOcc to Occ provi